##### [Retour au sommaire](../../index.ipynb)

# 4.2 Gestion des processus par le système d'exploitation

Pour étudier la gestion des processus par un système d'exploitation nous allons utiliser l'OS GNU/Linux.

Mais avant tout qu'est ce qu'un processus?

**Définitions**

<div class="alert alert-info">
    Un <b>processus</b> est un <b>programme en cours d'exécution</b>.
</div>


Pour fonctionner un processus nécessite:
1. Un **ensemble d'instructions** à executer par le processeur. Ces instructions sont, la plupart du temps, transférées de la mémoire de masse (disque dur) vers la mémoire vive (RAM);
2. Un **espace d'adressage en mémoire vive** pour trouver/écrire les **données** de travail;
3. Des **ressources** permettant les entrées sorties des données ( Disque dur, carte réseau, carte graphique, ... mais également fichiers, base de données...)

<div class="alert alert-info">
    Un <b>programme</b> est un fichier executable (le plus souvent écrit dans un langage haut niveau : java, python, haskell, C, C++) et compilé (ou interprété) pour l'écrire sous forme de langage machine pouvant être interprété par le processeur.
</div>

Le code source d'un même programme peut être utilisé sous **plusieurs architectures matérielles** grace au complilateur/interpréteur, par exemple gcc sous GNU/Linux.

Voici par exemple une [liste des architectures supportées par le noyau linux](https://en.wikipedia.org/wiki/List_of_Linux-supported_computer_architectures).

<div class="alert alert-info">
Il ne faut pas confondre le fichier executable (.exe sous Windows) avec le, ou les, processus que ce fichier va générer :

Un executable est un programme sous forme de fichier;

Un processus est une instance de ce programme associée aux ressources nécessaires à son fonctionnement.
</div>



**Le rôle du système d'exploitation**

Les systèmes d'exploitation modernes (GNU/Linux, iOS, Windows, Android...) sont appelés multi-tâches. Ils permettent de donner l'impression à l'utilisateur que plusieurs processus peuvent être executés en même temps. En réalité les processus sont exécutés chacun leur tour grace à un **système d'états**.

**Remarque**:
Si il y a plusieurs processeurs, l'exécution des processus est distribuée entre eux.

Un des rôles du système d'exploitation est de **créer**, **ordonnancer**, et **executer** les multiples processus.

## création d'un processus

Un processus est créé :
- au démarrage de l'OS
- par l'appel d'un autre processus
- par une action extérieure (utilisateur, réseau...)

sous GNU/Linux, la création d'un processus clone se fait par l'appel de **fork()** par un autre processus.
Si un processus A crée un processus B  et C par l'appel d'un fork alors

- A est le père de B
- A est le père de C
- B et C sont les fils de A

Les sous processus peuvent à leur tour créer des processus fils.
On peut modéliser ces relations par un **arbre**.

![process_tree](img/process_tree.jpg)

Les processus sont identifiés par leur **PID** (Process ID). Le PID est envoyé au processus père afin qu'il puisse communiquer avec son fils, et chaque processus possède l'identifiant de son père le **PPID** (Parent Process ID).


Mais qui est le premier processus ?

Avant le lancement du système, le chargeur d'amorce ( MBR : Master Boot Record) démarre le noyau linux avec la fonction start_kernel().
Cette fonction va créer le tout premier processus, le swapper (c'est le processus 0), il n'est le fils d'aucun processus.
Ce processus, avant de s'endormir, crée à son tour:
- le processus 1 (init ou systemd) qui va créer les premiers processus nécessaires au fonctionnement du système.
- le processus 2 ( [kthreadd] ) qui va gérer la partie matérielle

![2 premiers processus](img/graphe-hierarchie-up-p.jpg)


**Remarque**

les processus de PID 1 et 2 ont pour PPID 0.

### les processus de l'espace utilisateur

Les processus enfants issus de init concernent l'**espace utilisateur** du système.
On y trouve, entre autres:
- les processus bas niveaux
- les processus des programmes
- les processus des demons ( tâche qui tourne en tâche de fond, reconnaissable par un nom qui se termine par d)

![processus espace utilisateur](img/graphe-hierarchie-user-p.jpg)

### les processus du noyau

Les processus enfants issus de kthreadd concernent l'**espace noyau**.
Tous les processus de cet espace ont pour père kthreadd, ils ont tous comme PPID 2.
Tous les noms des processus de cet espace sont entourés par des crochets.

![processus espace noyau](img/graphe-hierarchie-noyau-p.jpg)

### L'interface vers le noyau

le dossier /proc est un pseudo-système de fichier qui sert d'interface avec le noyau.
Ainsi il est possible de lire des informations sur un processus depuis ce système de fichier.

Les informations disponibles sur un processus peuvent être lues depuis l'arborescence /proc/<PID du process>

## Commandes utiles
Plusieurs commandes linux utilisent cette interface vers le noyau afin de visualiser l'état des processus
Parmis ces commandes on trouve:
- ps
- pstree
- top

## Les différents états d'un processus

Les états standards d'un processus sont:

- Initialisation (new) : C'est le premier état d'un processus. Il attend que l'ordonnanceur le place dans l'état suivant (prêt)
- Prêt (ready) : Dans cet état le processus est chargé en mémoire centrale et attend son execution par le processeur. Les processus sont rangés dans une file qui est gérée par l'ordonnanceur.
- Elu (Running) : le processus est en cours d'exécution par le processeur
- Bloqué/Endormi (Sleeping) : un processus élu peut passer dans cet état car le processeur doit attendre un événement ( lecture de disque dur, réponse réseau...)
- Terminé (terminated) : le programme est terminé (le résultat est connu ou le programme a été forcé de s'arrêter)

Voici le diagramme d'états-transitions d'un processus

![diagramme états-transitions d'un processus](img/250px-Diagramme_etat_processus.png)

<div class="alert alert-info">
    Seul le diagramme ci-dessus est à connaitre.
</div>


Un système simple coeur ne peut exécuteur qu'une instruction à la fois. Il ne peut y avoir qu'un seul processus qui se trouve dans l'état élu, les autres processus doivent attendre leur tour pour être exécutés. Les transitions entre les différents états est assuré ar l'**ordonnanceur** (scheduleur), un de ses rôles est de déterminer qui, parmi les processus prêts, va passer à l'état élu.

**Pour information uniquement**

Il existe une granulométrie plus fine des états dans le système linux.

               D    uninterruptible sleep (usually IO)
               R    running or runnable (on run queue)
               S    interruptible sleep (waiting for an event to complete)
               T    stopped by job control signal
               Z    defunct ("zombie") process, terminated but not reaped by its parent

et voici un diagramme plus réaliste des états.

![diagramme états-transitions d'un processus](img/diagram_process_complet.jpg)

<div class="alert alert-info">
    Le diagramme ci-dessus est uniquement pour information, il n'est pas à connaitre.
</div>
 
 et il en existe des  [plus complexes](https://media.geeksforgeeks.org/wp-content/uploads/20190604122001/states_modified.png)...

## ordonnancement des processus

L'ordonnanceur du systême d'exploitation est un élément critique car la réactivité du système va dépendre de lui.

On peut envisager **plusieurs politiques d'ordonnancement**:
 
 - une file : le premier arrivé est le premier servi : très simple mais vraiment pas efficace
 - plus court d'abord : difficile à mettre en oeuvre car la durée d'execution est rarement connue
 - priorité : l'OS alloue une priorité
 - tourniquet : un temps est alloué à chaque processus. Si le processus n'est pas terminé il passe à l'état ready.

<div class="alert alert-info">Dans le système Unix standard la solution est un système hybride entre tourniquet (Round-robin) et priorité.</div> 

On peut dire qu'il existe deux grands types d'algorithmes d'ordonnancement:

 1. L'ordonnancement qui partage le temps : il est présent sur les ordinateurs classiques. Pour une station de travail, l'ordonnancement va prioriser les processus qui interagissent avec l'utilisateur. Pour des stations de calculs les temps alloués seront partagés plus équitablement entre chaque processus.
 2. L'ordonnancement **temps réel** qui permet d'assurer un temps d'execution dans un délai donné.
 
Evidemment tout ceci est configurable... 

Sous GNU/Linux il est possible de dialoguer avec l'ordonnanceur en fixant des priorités différentes aux processus.

- la commande **chrt** permet de changer la politique d'ordonnancement d'un processus donné.
- la commande **nice** permet de changer la priorité d'un processus donné.

## L'interblocage

(deadlock en anglais)

Imaginons deux processus et deux ressources (accès au disque dur et accès à une adresse mémoire).

Voici le scénario:

- Le processus 1 passe à l'état **élu**, puis demande la ressource 1. l'OS lui fournit et le passe à l'état **prêt**.
- Le processus 2 passe à l'état **élu**, puis demande la ressource 2. l'OS lui fournit et le passe à l'état **prêt**.
- L'OS repasse le processus 1 a l'état **élu**. Il demande la ressource 2 qui est assignée au processus 2. L'OS le met donc à l'état **bloqué**.
- L'OS repasse le processus 2 a l'état **élu**. Il demande la ressource 1 qui est assignée au processus 1. L'OS le met donc à l'état **bloqué**.


Nous allons représenter ce scénario par un graphe orienté avec la convention suivante:

- Si un processus Pi demande une ressource Ri nous allons symboliser cette relation par une arête **orientée de Pi vers Ri**
- Si une ressource Ri est assignée à un processus Pi nous allons symboliser cette relation par une arête **orientée de Ri vers Pi**

Le graphe du scénario peut se représenter par:

![](img/deadlock1.png)

En modifiant **uniquement les emplacements des processus et des ressources** on obtient ce graphe:

![](img/deadlock2.png)

<div class="alert alert-info">
    La mise en évidence d'un <b>cycle</b> dans le graphe montre que nous sommes dans une situation d'interblocage.
</div>
On se retrouve dans la situation de quatre automobilistes sur un carrefour avec priorité à droite...

### Les conditions nécessaires à l'interblocage

- Exclusion mutuelle : les ressources ne sont pas partageables (ce qui est le cas par exemple pour l'accès à un disque dur, à une carte réseau...)
- Possession et attende : Il doit existe un processus qui utilise une ressource également en attente
- Absence de préemption : Il n'y a pas d'acteur tiers qui permettent de libérer la ressource (les libérations sont faites par les processus)
- Attente circulaire : Il doit exister un processus Pi qui attend une ressource possédée par P{i+1}




## Webographie
- [Processus Wikipedia](https://fr.wikipedia.org/wiki/Processus_(informatique))
- [lecluse.fr](https://www.lecluse.fr/nsi/NSI_T/archi/process/)
- [nsi4noobs](http://nsi4noobs.fr/Les-processus-informatiques)
- [it-connect.fr](https://www.it-connect.fr/les-processus-sous-linux/)
- [école polytechnique de Montréal]'https://cours.polymtl.ca/inf2610/documentation/notes/chap7.pdf)

[Retour au sommaire](../../index.ipynb)